# Enrichment analysis

In [1]:
# Import modules
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact

### Functions

In [2]:
# Get proportions
def get_proportions(dataset, feature):
    "Gets the proportion of a specific feature in a given dataset"
    total_length = dataset.shape[0]
    
    if feature == 'Cofactor':
        sset = dataset[(dataset['Cofactor'].notna()) &
                      ~(dataset["Cofactor"].str.contains(r"\d+Fe-\d+S", regex=True, na=False))]
        
    elif feature == 'Susceptible cys':
        sset = dataset[(dataset['cys in AS'] > 0) |
                       (dataset['cys in BS'] > 0)]
        
    elif feature == 'FeS clusters':
        sset = dataset[dataset["Cofactor"].str.contains(r"\d+Fe-\d+S", regex=True, na=False)]
        
    else:
        sset = dataset[dataset[feature] > 0]
    
    pos_length = sset.shape[0]
    neg_length = total_length - pos_length
    
    return pos_length, neg_length

### P.putida evidence data

In [3]:
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

In [4]:
ros_table = pd.read_csv('ROS_summary_BRENDA_full.tsv', sep = '\t', encoding = 'latin1')
ros_length = ros_table.shape[0]

In [5]:
evidence_table = pd.read_csv('Pputida_evidenced.txt', sep = '\t', encoding = 'latin1')

no_evidence_table = ros_table[~ros_table["UniProt ID"].isin(evidence_table["UniProt ID"])]
print(no_evidence_table.shape)

evidence_length = evidence_table.shape[0]
no_evidence_length = no_evidence_table.shape[0]

(968, 20)


In [6]:
classes = ['Oxidoreductases', 'Transferases', 'Hydrolases', 'Lyases', 'Isomerases', 'Ligases', 'Translocases']

__1) Enrichment of each Structural Feature within Evidenced and full dataset__

In [7]:
features = ['Cofactor', 'FeS clusters', '# SR in AS', 'Susceptible cys', '# SR in BS', '# Disulfide bonds']

enrich_analysis_results = []

for feature in features:
    a, b = get_proportions(evidence_table, feature)
    c, d = get_proportions(no_evidence_table, feature)
    cont_table = np.array([[a, b],[c,d]])  # Contingency table
    
    oddsratio, pvalue = fisher_exact(cont_table, alternative = "greater")

    # Store results
    enrich_analysis_results.append({
            "Feature": feature,
            "evidenced": a,
            "no evidenced": c,
            "oddsratio": float(oddsratio),
            "pvalue": pvalue
        })

enrich_df = pd.DataFrame(enrich_analysis_results)

# Apply FDR correction
enrich_df["p_adj"] = multipletests(
    enrich_df["pvalue"],
    method = "fdr_bh"
    )[1]

# Filter significant enrichments
significant = enrich_df[enrich_df["p_adj"] <= 0.05]
# display(significant)

filename = 'SF_evidences.csv'
enrich_df.to_csv(f'./Tablas_enrichment/{filename}', sep = '\t', index = False)

There is a clear evidence of enrichment of each analyzed Structural Featurer in Evidenced data

__2) Enrichment of each Structural Feature between proteins from a specific EC class and all others__

In [9]:
# Enrichment analysis for each class (evidence-independent)
features = ['Cofactor', 'FeS clusters', '# SR in AS', 'Susceptible cys', '# SR in BS', '# Disulfide bonds']
print('Enrichment analysis of Structural Features between a specific EC class and all other classes')

for enz_class in classes:
    enrich_analysis_results = []
    pos_sset = ros_table[ros_table["EC class"] == enz_class]
    pos_sset_length = pos_sset.shape[0]
    neg_sset = ros_table[ros_table["EC class"] != enz_class]
    neg_sset_length = neg_sset.shape[0]
    
    for feature in features:
        a, b = get_proportions(pos_sset, feature)
        c, d = get_proportions(neg_sset, feature)
        cont_table = np.array([[a, b],[c,d]])  # Contingency table

        oddsratio, pvalue = fisher_exact(cont_table, alternative = "greater")

        # Store results
        enrich_analysis_results.append({
                "Feature": feature,
                "evidenced": a,
                "all data": c,
                "oddsratio": oddsratio,
                "pvalue": pvalue
            })

    enrich_df = pd.DataFrame(enrich_analysis_results)

    # Apply FDR correction
    enrich_df["p_adj"] = multipletests(
        enrich_df["pvalue"],
        method = "fdr_bh"
        )[1]

    # Filter significant enrichments
    # significant = enrich_df[enrich_df["p_adj"] <= 0.05]
    # if not significant.empty:
    #    print('\n', enz_class.upper())
    #    display(significant)
    
    print('\n', enz_class.upper())
    
    filename = f'SF_EC_comp_{enz_class}.csv'
    enrich_df.to_csv(f'./Tablas_enrichment/{filename}', sep = '\t', index = False)
    
    display(enrich_df)

Enrichment analysis of Structural Features between a specific EC class and all other classes

 OXIDOREDUCTASES


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,35,74,2.184544,0.000529,0.001586
1,FeS clusters,6,14,1.811791,0.172832,0.172832
2,# SR in AS,47,92,2.488763,0.000009,0.000053
3,Susceptible cys,27,81,1.452381,0.076868,0.115301
4,# SR in BS,79,235,1.675055,0.001297,0.002594
5,# Disulfide bonds,6,14,1.811791,0.172832,0.172832



 TRANSFERASES


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,8,101,0.256678,0.999995,1.000000
1,FeS clusters,3,17,0.626999,0.847205,1.000000
2,# SR in AS,33,106,1.135405,0.312172,0.624344
3,Susceptible cys,30,78,1.437247,0.074376,0.223127
4,# SR in BS,79,235,1.318455,0.051287,0.223127
5,# Disulfide bonds,0,20,0.000000,1.000000,1.000000



 HYDROLASES


...

,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,6,103,1.159630,0.440021,0.660031
1,FeS clusters,0,20,0.000000,1.000000,1.000000
2,# SR in AS,13,126,2.387346,0.011418,0.068509
3,Susceptible cys,4,104,0.730769,0.791232,0.949478
4,# SR in BS,20,294,1.559934,0.092042,0.276126
5,# Disulfide bonds,2,18,2.224586,0.253386,0.506772



 LIGASES


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,10,99,1.412458,0.214991,0.429983
1,FeS clusters,0,20,0.000000,1.000000,1.000000
2,# SR in AS,8,131,0.794878,0.775791,1.000000
3,Susceptible cys,13,95,2.023823,0.028644,0.085932
4,# SR in BS,36,278,2.513754,0.000198,0.001188
5,# Disulfide bonds,0,20,0.000000,1.000000,1.000000



 TRANSLOCASES


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,0,109,0.000000,1.000000,1.000000
1,FeS clusters,0,20,0.000000,1.000000,1.000000
2,# SR in AS,0,139,0.000000,1.000000,1.000000
3,Susceptible cys,3,105,1.685714,0.302105,0.629841
4,# SR in BS,7,307,1.415754,0.314920,0.629841
5,# Disulfide bonds,1,19,3.006192,0.304999,0.629841


__3) Enrichment of each Structural Feature within Evidenced and No Evidenced data, divided by protein class (EC)__

In [10]:
print('Enrichment of each Structural Feature between Evidenced and No Evidenced data, divided by protein class (EC)')
for enz_class in classes:
    enrich_analysis_results = []
    pos_sset = evidence_table[evidence_table["EC class"] == enz_class]
    neg_sset = no_evidence_table[no_evidence_table["EC class"] == enz_class]
    
    pos_len = pos_sset.shape[0]
    neg_len = neg_sset.shape[0]
    
    if pos_sset.empty:
        continue
        
    for feature in features:
        a, b = get_proportions(pos_sset, feature)
        c, d = get_proportions(neg_sset, feature)
        cont_table = np.array([[a, b],[c,d]])  # Contingency table

        oddsratio, pvalue = fisher_exact(cont_table, alternative = "greater")

        # Store results
        enrich_analysis_results.append({
                "Feature": feature,
                "evidenced": a,
                "all data": c,
                "oddsratio": oddsratio,
                "pvalue": pvalue
            })

    enrich_df = pd.DataFrame(enrich_analysis_results)

    # Apply FDR correction
    
    enrich_df["p_adj"] = multipletests(
        enrich_df["pvalue"],
        method = "fdr_bh"
        )[1]

    # Filter significant enrichments
    # significant = enrich_df[enrich_df["p_adj"] <= 0.05]
    # if not significant.empty:
    #    print('\n', enz_class.upper())
    #    display(significant)
    
    filename = f'SF_evidence_{enz_class}.csv'
    enrich_df.to_csv(f'./Tablas_enrichment/{filename}', sep = '\t', index = False)
    
    print('\n', enz_class)
    display(enrich_df)

Enrichment of each Structural Feature between Evidenced and No Evidenced data, divided by protein class (EC)

 Oxidoreductases


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,14,21,10.000000,0.000002,0.000010
1,FeS clusters,1,5,1.443478,0.550237,0.550237
2,# SR in AS,13,34,4.762032,0.000656,0.001968
3,Susceptible cys,8,19,4.000000,0.007685,0.011527
4,# SR in BS,16,63,3.428571,0.005468,0.010937
5,# Disulfide bonds,2,4,3.795455,0.160081,0.192097



 Transferases


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,0,8,0.000000,1.000000,1.000000
1,FeS clusters,1,2,35.666667,0.053801,0.107601
2,# SR in AS,3,30,18.600000,0.011174,0.033523
3,Susceptible cys,4,26,inf,0.000289,0.001731
4,# SR in BS,3,76,5.526316,0.133232,0.199848
5,# Disulfide bonds,0,0,NaN,1.000000,1.000000



 Hydrolases


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,0,29,0.0,1.000000,1.000000
1,FeS clusters,0,0,NaN,1.000000,1.000000
2,# SR in AS,1,19,inf,0.169492,0.508475
3,Susceptible cys,1,10,inf,0.093220,0.508475
4,# SR in BS,0,39,0.0,1.000000,1.000000
5,# Disulfide bonds,0,4,0.0,1.000000,1.000000



 Lyases


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,0,14,0.000000,1.000000,1.000000
1,FeS clusters,5,6,36.666667,0.000110,0.000660
2,# SR in AS,1,17,0.754902,0.758604,1.000000
3,Susceptible cys,3,12,5.125000,0.064962,0.194887
4,# SR in BS,3,42,0.928571,0.681701,1.000000
5,# Disulfide bonds,0,2,0.000000,1.000000,1.000000



 Isomerases


,Feature,evidenced,all data,oddsratio,pvalue,p_adj
0,Cofactor,1,5,2.666667,0.417537,0.626305
1,FeS clusters,0,0,NaN,1.000000,1.000000
2,# SR in AS,3,10,10.500000,0.051969,0.103938
3,Susceptible cys,2,2,21.500000,0.028890,0.086669
4,# SR in BS,4,16,inf,0.022867,0.086669
5,# Disulfide bonds,0,2,0.000000,1.000000,1.000000


__¿Is there an enrichment of any enzyme class in the evidenced data?__

In [11]:
enrich_analysis_results = []

for enz_class in classes:
    no_sset = no_evidence_table[no_evidence_table["EC class"] == enz_class]
    no_num = no_sset.shape[0]
    no_prop = round(no_num / ros_length * 100, 2)

    yes_sset = evidence_table[evidence_table["EC class"] == enz_class]
    yes_num = yes_sset.shape[0]
    yes_prop = round(yes_num / evidence_length * 100, 2)

    print(f'Proportion of {enz_class} in evident: {yes_prop}%')
    print(f'Proportion of {enz_class} in all data: {no_prop}%')
    print('')

    # Chi^2
    a = yes_num
    b = evidence_length - yes_num
    c = no_num
    d = ros_length - no_num
    
    cont_table = np.array([[a, b],[c,d]])
    
    oddsratio, pvalue = fisher_exact(cont_table, alternative = "greater")

    # Store results
    enrich_analysis_results.append({
            "Enzyme class": enz_class,
            "evidenced": a,
            "no_evidenced": c,
            "oddsratio": oddsratio,
            "pvalue": pvalue
        })


enrich_df = pd.DataFrame(enrich_analysis_results)

# Apply FDR correction
enrich_df["p_adj"] = multipletests(
    enrich_df["pvalue"],
    method = "fdr_bh"
    )[1]

# Filter significant enrichments
# significant = enrich_df[enrich_df["p_adj"] <= 0.05]
# display(significant)

display(enrich_df)

filename = f'EC.csv'
enrich_df.to_csv(f'./Tablas_enrichment/{filename}', sep = '\t', index = False)

Proportion of Oxidoreductases in evident: 60.0%
Proportion of Oxidoreductases in all data: 16.96%

Proportion of Transferases in evident: 10.0%
Proportion of Transferases in all data: 21.43%

Proportion of Hydrolases in evident: 2.5%
Proportion of Hydrolases in all data: 11.61%

Proportion of Lyases in evident: 17.5%
Proportion of Lyases in all data: 9.33%

Proportion of Isomerases in evident: 10.0%
Proportion of Isomerases in all data: 4.46%

Proportion of Ligases in evident: 0.0%
Proportion of Ligases in all data: 6.94%

Proportion of Translocases in evident: 0.0%
Proportion of Translocases in all data: 1.79%



,Enzyme class,evidenced,no_evidenced,oddsratio,pvalue,p_adj
0,Oxidoreductases,24,171,7.342105,3.346520e-09,2.342564e-08
1,Transferases,4,216,0.407407,9.813785e-01,1.000000e+00
2,Hydrolases,1,117,0.195266,9.923665e-01,1.000000e+00
3,Lyases,7,94,2.062540,8.148927e-02,2.607947e-01
4,Isomerases,4,45,2.377778,1.117691e-01,2.607947e-01
5,Ligases,0,70,0.000000,1.000000e+00,1.000000e+00
6,Translocases,0,18,0.000000,1.000000e+00,1.000000e+00


Only oxidoreductases are enriched in the evidenced set